# 인공지능 과제6
### : CRFs 기반 자동 띄어쓰기 성능 향상
- 학번: 201711719
- 학과: 응용통계학과
- 이름: 심은선
- 제출날짜: 2020.10.19

## 0. Install & Load libraries

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [2]:
!pip install sklearn-crfsuite

In [3]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split

## 1. Data load

In [4]:
def read_file(file_path):
  # "spacing_data.txt" 파일을 읽고 lines에 읽은 데이터를 저장
  with open(file_path, "r", encoding="utf8") as inFile:
      lines = inFile.readlines()

  # 데이터를 음절로 이루어진 문장과 정답 값으로 나누어 저장
  datas = []
  for line in lines:
      pieces = line.strip().split("\t")
      eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
      datas.append((eumjeol_sequence, label))

  return datas

In [5]:
# 파일 경로
file_path = "/gdrive/My Drive/인공지능(4-2)/week6_실습/spacing_data.txt"

datas = read_file(file_path)
X, y = zip(*datas)

## 2. Train test split

In [6]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.1, random_state=42)  
train_datas, test_datas = list(zip(train_x, train_y)), list(zip(test_x, test_y))

print("train_datas 개수 : " + str(len(train_datas)))          
print("test_datas 개수 : " + str(len(test_datas)) + "\n")

print("X:", train_datas[0][0])
print("y:", train_datas[0][1])

train_datas 개수 : 900
test_datas 개수 : 100

X: ['몽', '골', '사', '람', '의', '고', '대', '종', '교', '또', '한', '우', '리', '와', '같', '은', '무', '교', '(', '巫', '敎', ')', '로', '서', '하', '늘', '을', '최', '고', '의', '신', '으', '로', '섬', '겼', '다', '.']
y: ['B', 'I', 'B', 'I', 'I', 'B', 'I', 'B', 'I', 'B', 'I', 'B', 'I', 'I', 'B', 'I', 'B', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'I']


## 3. Preprocessing

In [7]:
# CRFs에 학습시키기 위해 음절을 feature화 한다
def sent2feature(eumjeol_sequence):
  features = []
  for idx in range(len(eumjeol_sequence)):
    feature = dict()
    if idx != 0:
      feature["PREV"] = eumjeol_sequence[idx-1] 
    if idx != len(eumjeol_sequence)-1:
      feature["NEXT"] = eumjeol_sequence[idx+1]
    feature["CUREENT"] = eumjeol_sequence[idx]
    features.append(feature)
  return features

In [8]:
train_x = [sent2feature(x) for x in train_x]
test_x = [sent2feature(x) for x in test_x]

In [9]:
print(train_x[0])
print(test_x[0])

[{'NEXT': '골', 'CUREENT': '몽'}, {'PREV': '몽', 'NEXT': '사', 'CUREENT': '골'}, {'PREV': '골', 'NEXT': '람', 'CUREENT': '사'}, {'PREV': '사', 'NEXT': '의', 'CUREENT': '람'}, {'PREV': '람', 'NEXT': '고', 'CUREENT': '의'}, {'PREV': '의', 'NEXT': '대', 'CUREENT': '고'}, {'PREV': '고', 'NEXT': '종', 'CUREENT': '대'}, {'PREV': '대', 'NEXT': '교', 'CUREENT': '종'}, {'PREV': '종', 'NEXT': '또', 'CUREENT': '교'}, {'PREV': '교', 'NEXT': '한', 'CUREENT': '또'}, {'PREV': '또', 'NEXT': '우', 'CUREENT': '한'}, {'PREV': '한', 'NEXT': '리', 'CUREENT': '우'}, {'PREV': '우', 'NEXT': '와', 'CUREENT': '리'}, {'PREV': '리', 'NEXT': '같', 'CUREENT': '와'}, {'PREV': '와', 'NEXT': '은', 'CUREENT': '같'}, {'PREV': '같', 'NEXT': '무', 'CUREENT': '은'}, {'PREV': '은', 'NEXT': '교', 'CUREENT': '무'}, {'PREV': '무', 'NEXT': '(', 'CUREENT': '교'}, {'PREV': '교', 'NEXT': '巫', 'CUREENT': '('}, {'PREV': '(', 'NEXT': '敎', 'CUREENT': '巫'}, {'PREV': '巫', 'NEXT': ')', 'CUREENT': '敎'}, {'PREV': '敎', 'NEXT': '로', 'CUREENT': ')'}, {'PREV': ')', 'NEXT': '서', 'CUREENT': '로'}, 

In [10]:
print(len(train_x))
print(len(test_x))

900
100


## 4. Training CRFs

In [11]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs')
crf.fit(train_x, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

## 5. Visualize & Evaluation

In [12]:
# 띄어쓰기 label, predict 결과를 시각화한다.
def show_predict_result(test_datas, predict):
  for index_1 in range(len(test_datas)):
      eumjeol_sequence, correct_labels = test_datas[index_1]
      predict_labels = predict[index_1]
     
      correct_sentence, predict_sentence = "", ""
      for index_2 in range(len(eumjeol_sequence)):
          if(index_2 == 0): # 문장의 시작 음절은 그대로 넣어준다
              correct_sentence += eumjeol_sequence[index_2]
              predict_sentence += eumjeol_sequence[index_2]
              continue

          if(correct_labels[index_2] == "B"):
              correct_sentence += " "
          correct_sentence += eumjeol_sequence[index_2]

          if (predict_labels[index_2] == "B"):
              predict_sentence += " "
          predict_sentence += eumjeol_sequence[index_2]

      print("정답 문장 : " + correct_sentence)
      print("출력 문장 : " + predict_sentence)
      print()

In [13]:
# Prediction
predict = crf.predict(test_x)

In [14]:
# Print Accuracy
print("Accuracy score : " + str(metrics.flat_accuracy_score(test_y, predict)))

Accuracy score : 0.9286721172822103


In [15]:
# Visualize
print("5개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

show_predict_result(test_datas[:5], predict[:5])

5개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장 : 남편마저 생활 능력이 없는 데다 술을 좋아하여 부부 싸움이 끊일 날이 없었는데 준호가 중학교에 입학하던 해 남편은 제 발로 집을 나갔다.
출력 문장 : 남편마 저생 활능력이 없는데 다술을 좋아하여부부싸움이 끊일날이 없었는데 준호가 중학교에 입학하던 해 남편은 제발로 집을 나갔다.

정답 문장 : 이루에는 몽고리(蒙古里), 몽골(蒙骨), 몽고사(蒙古斯), 맹고(盟古), 맹골(盟骨), 맹골자(盟骨子) 따위로 불렀다.
출력 문장 : 이루에는 몽고리(蒙古里), 몽골(蒙骨), 몽고 사(蒙古斯), 맹고(盟古), 맹골(盟骨), 맹골자(盟 骨子) 따위로 불렀다.

정답 문장 : 정식 국명이 '몽골 공화국'인 데다가 저들 자신도 몽골로 불리기를 바라기 때문이다.
출력 문장 : 정식 국명이 '몽골 공화국'인데다가 저들 자신도 몽골로 불리기를 바라기 때문이다.

정답 문장 : 전혀 제물 때문인 것입니다.
출력 문장 : 전혀 제물 때문인 것입니다.

정답 문장 : 전화벨이 울렸다.
출력 문장 : 전화벨이 울렸다.

